In [1]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Anticipatory Model  

In this file we develop the bidding model for an anticipatory agent. We mainly consider the following implications

* Adjusted flow
* Power Swap
* Overnomination

Our Nomination makes a difference on how much water can be discharged in the next step. On the other hand, we receive or have to deliver a power swap depending on the discrepancy between nomination and adjusted flow.  
The following formulas are directly included into the problem formulation:

$$
\begin{align*}
Q^\text{adj}_{r} &= \frac{Q^\text{nom}_{j,r} \cdot p_{j,r} + Q^\text{nom}_{O,r} \cdot p_{O,r}}{p_{j,r} + p_{O,r}} \\
P^\text{swap}_{j,r} &= p_{j,r} \cdot (Q^\text{nom}_r - Q^\text{adj}_r) - \sum\limits_{k \in \mathcal{K}_{O,r}} P^\text{Over}_k \\
P^\text{Over}_k  &= \max \{0, e_k \cdot (Q^\text{adj}_r - Q^\text{spill}_k)\} 
\end{align*}
$$

The Power Swap has to be delivered smoothly in the last 12 hours of the day. The Power Swaps in total add up and we have the following obligation:
$$
\begin{alignat*}{3}
y_{t} &= \sum\limits_{k \in \mathcal{K}} w_{k,t} + z^+_{t} - z^-_{t}  & & \text{ if } \; t = 1, \ldots, 12 \\ 
y_{t} &= \sum\limits_{k \in \mathcal{K}} w_{k,t} + 2 \cdot \sum\limits_{r \in \mathcal{R}} P^\text{swap}_r + z^+_{t} - z^-_{t} & & \text{ if } \; t = 13, \ldots, 24
\end{alignat*}
$$

Additionally, the adjusted flow poses the real average discharge constraint:
$$
T \cdot Q^\text{adj}_r = \sum\limits_{t = 1}^T Q^\text{real}_{r,t}
$$

### Define Parameters and Aggregated Other

We have to define an aggregated other user $O$ to reflect some constraints in our program.

In [22]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
R, plants, parts = read_data(filepath_Ljungan)
print() 

j = parts[1]
O = OtherParticipant(parts, j , R)[1]
K_j = [j.plants[1]]
K_O = [O.plants[1]]
pj = j.participationrate
pO = O.participationrate


println("Participation rate $(j.name): \n ", pj)
println("Participation rate $(O.name): \n ", pO)
println("K_O : $(K_O) and spillage $(K_O[1].spillreference), \n K_j : $(K_j) and spillage $(K_j[1].spillreference)")

Participation rate Sydkraft: 
 ________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Participation rate Other: 
 ________________________________
Flasjon  | 2.68    
Holsmjon | 2.68    

K_O : HydropowerPlant[Parteboda] and spillage 1.4, 
 K_j : HydropowerPlant[Flasjo] and spillage 0.58


In [23]:
Stages = 2 # 2 lowest number as first stage is just to achieve nonanticipativity
T = 24
PPoints = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
I = length(PPoints)-1

println("Stages :" , Stages, "\n Hours: ", T, "\n I : ", I, "\n Price Points = ", [i for i in PPoints])

Qref = Dict{Reservoir, Float64}(r => 10.0 for r in R)
scenario_count = 3
Prices = [floor.(rand(T), sigdigits=3) for i in 1:scenario_count]
Inflows = [10.0]
Omega = [(price = p, inflow = v, nomination = 0.3) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Costs
S = 0.1
# Cost for Up and Down Balancing
mu_up = 1
mu_down = 0.3

println("Uncertainty Set: ", Omega)
println(Qref)
println(Prices)

Stages :8
 Hours: 24
 I : 5
 Price Points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]


Uncertainty Set: NamedTuple{(:price, :inflow, :nomination), Tuple{Vector{Float64}, Float64, Float64}}[(price = [0.61, 0.975, 0.805, 0.957, 0.6, 0.554, 0.23, 0.368, 0.412, 0.614, 0.24, 0.831, 0.158, 0.615, 0.491, 0.0277, 0.634, 0.254, 0.872, 0.786, 0.11, 0.751, 0.306, 0.429], inflow = 10.0, nomination = 0.3), (price = [0.765, 0.254, 0.474, 0.936, 0.282, 0.652, 0.878, 0.53, 0.454, 0.99, 0.473, 0.905, 0.985, 0.768, 0.986, 0.908, 0.142, 0.0223, 0.599, 0.303, 0.228, 0.432, 0.0847, 0.837], inflow = 10.0, nomination = 0.3), (price = [0.755, 0.387, 0.798, 0.888, 0.333, 0.866, 0.363, 0.302, 0.229, 0.0127, 0.188, 0.247, 0.787, 0.103, 0.609, 0.943, 0.614, 6.55e-5, 0.448, 0.00152, 0.432, 0.547, 0.26, 0.665], inflow = 10.0, nomination = 0.3)]
________________________________
Flasjon  | 10.0    
Holsmjon | 10.0    

[[0.61, 0.975, 0.805, 0.957, 0.6, 0.554, 0.23, 0.368, 0.412, 0.614, 0.24, 0.831, 0.158, 0.615, 0.491, 0.0277, 0.634, 0.254, 0.872, 0.786, 0.11, 0.751, 0.306, 0.429], [0.765, 0.254, 0.474

In [24]:
function subproblem_builder_anticipatory(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= sum(k.equivalent * k.spillreference for k in K_j), SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l[r = R] <= r.maxvolume, SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, lind[r = R], SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, u_start[k = K_j], SDDP.State, initial_value = 0, Bin)
    @variable(subproblem, 0 <= Qnom[r = R] <= min([k.spillreference for k in filter(k -> k.reservoir == r, plants)]...), SDDP.State, initial_value = 0)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, d[t=1:T, k = K_j], Bin)
    @variable(subproblem, u[t=1:T, k = K_j], Bin)
    @variable(subproblem, BALANCE_INDICATOR[r = R], Bin)
    @variable(subproblem, 0 <= w[t=1:T, k = K_j] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t=1:T] >= 0)
    @variable(subproblem, z_down[t=1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t=1:T, k = K_j] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t=1:T, r = R])
    @variable(subproblem, 0 <= Qadj[r = R])
    @variable(subproblem, Pswap[r = R])
    @variable(subproblem, Pover[k = K_O] >= 0)
    @variable(subproblem, f[r = R] >= 0)
    @variable(subproblem, s[r = R] >= 0)
    @variable(subproblem, a[r = R])

    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    @constraint(subproblem, balance_ind[r = R], lind[r].out == lind[r].in - T * (Qnom[r].out - Qref[r])- s[r]) 
    @constraint(subproblem, nbal1[r = R], BALANCE_INDICATOR[r] => {Qnom[r].out <= Qref[r]})
    @constraint(subproblem, nbal2[r = R], !BALANCE_INDICATOR[r] => {0 <= lind[r].in})
    @constraint(subproblem, NoSpill[k = K_O], BALANCE_INDICATOR[k.reservoir] => {sum(Qnom[r_up].out for r_up in find_us_reservoir(k.reservoir)) <= k.spillreference})
    @constraint(subproblem, watervalue[r = R], a[r] >= pj[r] * 0.01 *  (l[r].in - l[r].out))
    if node == 1
        @stageobjective(subproblem, -sum(a[r] for r in R))
        @constraint(subproblem, balance_transfer[r = R], l[r].out == l[r].in - T * Qnom[r].out - s[r]) 
    else
        @constraint(subproblem, adjustedflow[r = R], (pj[r] + pO[r]) * Qadj[r] - Qnom[r].in * pj[r] ==  pO[r])
        @constraint(subproblem, powerswap[r = R], Pswap[r] == pj[r] * (Qnom[r].in - Qadj[r]) - sum(Pover[k] for k in K_O))
        @constraint(subproblem, overnomination[k = K_O], Pover[k] >= k.equivalent * (Qadj[k.reservoir] - k.spillreference))
        @constraint(subproblem, endcond[k = K_j], u_start[k].out == u[T,k])
        @constraint(subproblem, startcond[k = K_j], u_start[k].in == u[1,k])
        @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
        @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
        @constraint(subproblem, obligation[t=1:T], y[t] - sum(Pswap[r] for r in R) == sum(w[t,k] for k in K_j) + z_up[t] - z_down[t])
        @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qadj[r] + f[r] * T - s[r])
        @constraint(subproblem, active[t=1:T, k=K_j], w[t,k] <= u[t,k] * k.spillreference * k.equivalent)
        @constraint(subproblem, startup[t=1:T-1, k=K_j], d[t,k] >= u[t+1,k] - u[t,k])
        @constraint(subproblem, production[t=1:T, k=K_j], w[t,k] == Qeff[t,k] * k.equivalent)
        @constraint(subproblem, realwater[t=1:T, k=K_j], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
        @constraint(subproblem, spillwater[t=1:T, k=K_j], Qeff[t,k] <= k.spillreference)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            for r in R
                JuMP.fix(f[r], om.inflow, force=true)
                JuMP.set_normalized_rhs(adjustedflow[r], pO[r] * om.nomination)
            end
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * z_up[t] + mu_down * z_down[t]  - S * sum(d[t,k] for k in K_j) for t in 1:T) - sum(a[r] for r in R))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder_anticipatory (generic function with 1 method)

In [25]:
model_ant = SDDP.LinearPolicyGraph(
    subproblem_builder_anticipatory;
    stages = Stages,
    sense = :Max,
    upper_bound = Stages * T * sum(k.equivalent * k.spillreference for k in plants),
    optimizer = CPLEX.Optimizer
)

SDDP.train(model_ant; iteration_limit = 10)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 151
  scenarios       : 2.18700e+03
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [532, 532]
  Vector{AffExpr} in MOI.Indicator{MOI.ACTIVATE_ON_ONE, MOI.LessThan{Float64}}  : [3, 3]
  VariableRef in MOI.LessThan{Float64}                                          : [197, 197]
  VariableRef in MOI.ZeroOne                                                    : [51, 51]
  AffExpr in MOI.LessThan{Float64}                                              : [120, 192]
  AffExpr in MOI.GreaterThan{Float64}                                           : [2, 26]

numerical stability report
  matrix range     [3e-04, 2e+01]
  objective range  [7e-05, 1e+00]
  bounds range     [2e-01, 4e+04]
  rhs range        [4e-01, 2e+02]
  - objective range contains small coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1  -1.612582e+01  1.024611e+02  6.070001e-01        30   1


         2   5.124005e+01  5.726601e+01  1.613000e+00        60   1


         3   5.737563e+01  5.726601e+01  2.446000e+00        90   1


...


         6   5.603019e+01  5.726601e+01  4.568000e+00       180   1


         7   5.757648e+01  5.726601e+01  5.690000e+00       210   1


         8   5.747563e+01  5.726601e+01  6.429000e+00       240   1


         9   5.716120e+01  5.726601e+01  7.257000e+00       270   1


        10   5.664591e+01  5.726601e+01  8.006000e+00       300   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 8.006000e+00
total solves   : 300
best bound     :  5.726601e+01
simulation ci  :  4.919165e+01 ± 1.427522e+01
numeric issues : 0
-------------------------------------------------------------------



In [26]:
rule_ant = SDDP.DecisionRule(model_ant; node = 1)
solution = SDDP.evaluate(
    rule_ant;
    incoming_state = Dict(Symbol("l[$(r)]") => r.currentvolume for r in R),
    controls_to_record = [:Qnom, :x, :z_up],
)

(stage_objective = -0.2561279999999897, outgoing_state = Dict(Symbol("x[2,24]") => 0.0, Symbol("x[6,18]") => 0.1798, Symbol("x[4,13]") => 0.1798, Symbol("x[2,8]") => 0.1798, Symbol("Qnom[Flasjon]") => 0.58, Symbol("x[3,17]") => 0.1798, Symbol("x[3,7]") => 0.1798, Symbol("x[1,4]") => 0.0, Symbol("x[6,7]") => 0.1798, Symbol("x[2,4]") => 0.0…), controls = Dict{Symbol, AbstractArray}(:Qnom => 1-dimensional DenseAxisArray{SDDP.State{Float64},1,...} with index sets:
    Dimension 1, Reservoir[Flasjon, Holsmjon]
And data, a 2-element Vector{SDDP.State{Float64}}:
 SDDP.State{Float64}(0.0, 0.5800000000000018)
 SDDP.State{Float64}(0.0, 1.4), :z_up => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.1798) … SDDP.State{Float64}(0.0, 0.0) SD

In [27]:
unique(values(solution[2]))

9-element Vector{Float64}:
     0.0
     0.1798
     0.58
 17966.4
 18206.4
 34986.08
     1.4
     1.0
 35226.08